In [1]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset
import pandas as pd

/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
model_name = "dmis-lab/biobert-base-cased-v1.1"


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
save_dir = "../models/biobert"

In [6]:
tokenizer.save_pretrained(save_dir)

('../models/biobert/tokenizer_config.json',
 '../models/biobert/special_tokens_map.json',
 '../models/biobert/vocab.txt',
 '../models/biobert/added_tokens.json',
 '../models/biobert/tokenizer.json')

In [7]:
model.save_pretrained(save_dir)

In [3]:
model_path = "../models/biobert"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

/home/shiva.heydari/Desktop/cls_mdu/env/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [6]:
sample_data = {
  "text": "short merge text ...",
  "modality": "MR",
  "vendor": "GE",
  "series_type": "FLAIR",
  "plane": "AX",
  "acquisition": "2D",
  "body": "brain",
  "contrast": "with"
}

In [7]:
encoded_sample = {
  "text": "short merge text ...",
  "label_modality": 0,
  "label_vendor": 1,
  "label_series_type": 2,
  "label_plane": 0,
  "label_acquisition": 1,
  "label_body": 3,
  "label_contrast": 0,
}

In [20]:
df_sample = pd.DataFrame([encoded_sample])
print(df_sample)

                   text  label_modality  label_vendor  label_series_type  \
0  short merge text ...               0             1                  2   

   label_plane  label_acquisition  label_body  label_contrast  
0            0                  1           3               0  


In [ ]:
type(df_sample["text"].tolist())
type(df_sample["text"])

pandas.core.series.Series

In [ ]:
#pytorch dataset for Dataset for multi-head labesl

class DicomText(Dataset):
    def __init__(self, df, tokenizer, max_length, label_maps):

        self.texts = df["text"].tolist()
        self.labels = df[ ["label_modality",
            "label_vendor",
            "label_series_type",
            "label_plane",
            "label_acquisition",
            "label_body",
            "label_contrast",
        ]].to_dict(orient="list")

        self.tokenizer = tokenizer
        self.max_length = max_length
        
    
        

    def __getitem__(self, idx):
        text = self.texts[idx]
        item = {"text": text}
        item["labels_modality"] = self.labels["label_modality"][idx]
        item["label_vendor"] = self.labels["label_vendor"][idx]
        item["label_series_type"] = self.labels["label_series_type"][idx]
        item["label_plane"] = self.labels["label_plane"][idx]
        item["label_acquisition"] = self.labels["label_acquisition"][idx]
        item["label_body"] = self.labels["label_body"][idx]
        item["label_contrast"] = self.labels["label_contrast"][idx]

        return item

    
    

    def __len__(self):
        return len(self.texts)

In [36]:
test = DicomText(df_sample)
test.labels

{'label_modality': [0],
 'label_vendor': [1],
 'label_series_type': [2],
 'label_plane': [0],
 'label_acquisition': [1],
 'label_body': [3],
 'label_contrast': [0]}

In [14]:
'''
input: raw record : dict 
output: tokenized 
'''



'\ninput: raw record : dict \noutput: tokenized \n'

In [ ]:
#model construction

In [ ]:
#evaluation